### Make sure you define the API tokens and board ID before running the code

## Part 1

![reference-monday-board](https://ik.imagekit.io/5jrct2yttdr/github/walton/Screenshot%202024-08-12%20205143_Q2tWQ3_vU.png?updatedAt=1723510349530)

In [3]:
import json
import os
from dataclasses import dataclass

import requests

In [1]:
# Define your API token and board ID
api_token = ""

# Define the board ID for the "Items Board"
board_id = 7212306580

# Setup the API endpoint
url = "https://api.monday.com/v2"

In [4]:
# Query the item IDs
query = """
query {
  boards (ids: %s){
    items_page {
      items {
        id
        name
      }
    }
  }
}
""" % (
    board_id
)

# Setup the headers with your API key
headers = {"Authorization": api_token, "Content-Type": "application/json"}

# Make the HTTP POST request to the API
response = requests.post(url, json={"query": query}, headers=headers)

# Parse the JSON response
response_data = json.loads(response.text)

# Initialize an empty list to store the IDs
item_ids = []

# Traverse the JSON structure to extract item IDs
for board in response_data['data']['boards']:
    for item in board['items_page']['items']:
        item_ids.append(item['id'])

# Print the list of item IDs
print(item_ids)

['7212306607', '7212306611', '7212306619']


In [5]:
# Convert the list of IDs to a string format suitable for the query
ids_string = ', '.join(item_ids)

# Construct the GraphQL query with the extracted IDs
query = f"""
query {{
  items (ids: [{ids_string}]) {{
    id,
    name,
    column_values {{
      id,
      text,
      column {{
        title
      }}
    }}
  }}
}}
"""

# Make the HTTP POST request to the API
response = requests.post(url, json={"query": query}, headers=headers)

# Print the response
response = json.loads(response.text)
print(json.dumps(response, indent=2))

{
  "data": {
    "items": [
      {
        "id": "7212306607",
        "name": "item1",
        "column_values": [
          {
            "id": "subitems__1",
            "text": null,
            "column": {
              "title": "Subitems"
            }
          },
          {
            "id": "text6__1",
            "text": "user1",
            "column": {
              "title": "name"
            }
          },
          {
            "id": "email__1",
            "text": "amalrajan74@gmail.com",
            "column": {
              "title": "email"
            }
          },
          {
            "id": "text__1",
            "text": "user1 content",
            "column": {
              "title": "email_content"
            }
          }
        ]
      },
      {
        "id": "7212306611",
        "name": "item2",
        "column_values": [
          {
            "id": "subitems__1",
            "text": null,
            "column": {
              "title": "Subitems"
   

In [6]:
# Define the dataclass to store the extracted information
@dataclass
class ItemInfo:
    item_id: str
    name: str
    email: str
    email_content: str


# Initialize an empty list to store the 2D array
result = []

# Iterate over each item
for item in response['data']['items']:
    item_id = item['id']
    name = item['name']
    email = None
    email_content = None
    
    # Extract email and email_content from column_values
    for column_value in item['column_values']:
        if column_value['column']['title'] == 'email':
            email = column_value['text']
        elif column_value['column']['title'] == 'email_content':
            email_content = column_value['text']
    
    # Create an instance of the dataclass and append it to the result list
    result.append(ItemInfo(item_id=item_id, name=name, email=email, email_content=email_content))

# Print the list of dataclass instances
for item_info in result:
    print(item_info)

ItemInfo(item_id='7212306607', name='item1', email='amalrajan74@gmail.com', email_content='user1 content')
ItemInfo(item_id='7212306611', name='item2', email='amalrajan@outlook.com', email_content='user2 content')
ItemInfo(item_id='7212306619', name='item3', email='amalrajan@cmail.carleton.ca', email_content='user3 content')


## Part 2

In [8]:
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

In [7]:
# Define the SendGrid API key
sendgrid_api_key = ""

In [9]:
def send_email(from_email, to_email, subject, content):
    message = Mail(
        from_email=from_email,
        to_emails=to_email,
        subject=subject,
        html_content=content)
    try:
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)

In [10]:
for item_info in result:
    send_email(
        "amalrajan74@gmail.com",
        item_info.email,
        item_info.name,
        item_info.email_content
    )


202
b''
Server: nginx
Date: Tue, 13 Aug 2024 00:40:25 GMT
Content-Length: 0
Connection: close
X-Message-Id: VzzHtmmWQwa1mAoCzmBjNg
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=600; includeSubDomains


202
b''
Server: nginx
Date: Tue, 13 Aug 2024 00:40:26 GMT
Content-Length: 0
Connection: close
X-Message-Id: 3J4p2LdnSFCT8cKU4SI99g
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=600; includeSubDomains


202
b''
Server: nginx
Date: Tue, 13 Aug 2024 00:40:2